# 🤖 01 - Data Exploration: Manipulator Health Monitoring (EDA)

This notebook marks **Phase 1: Exploratory Data Analysis (EDA)** of the manipulator (robot arm) degradation project.

Our goal is to thoroughly explore the raw data from the UR5 robot arm degradation dataset to understand **trends, missing values, and key correlations** among variables before proceeding with feature engineering and modeling.

---

## 🎯 Objective

Perform an Exploratory Data Analysis (EDA) on the **UR5 robot arm degradation dataset** (source: NIST).

The dataset consists of multiple CSV files containing controller-level sensing data (collected at 125Hz) under various operational conditions:
* **Temperature**
* **Payload**
* **Speed**

---

## 💾 Data Sources and Structure

The data files are organized within the project's `data/` directory.

### Key Data Files

| File Name | Format | Content Description |
| :--- | :--- | :--- |
| `UR5TestResult_header.xlsx` | Excel (`.xlsx`) | **Metadata** and **header information** for the sensor data. |
| `Calculated deviation of actual position to nominal position.xls` | Excel (`.xls`) | **Summary of pose accuracy degradation** (calculated deviation of actual position to nominal position). |
| `~18 CSV files` | CSV (`.csv`) | **Joint-level sensing data** across different test conditions (the primary data for analysis). |


In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# File handling
import os
import glob

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [3]:
# File paths
header_path = "../data/raw/header/ur5testresult_header.xlsx"
summary_path = "../data/raw/summary/calculated_deviation_of_actual_position_to_nominal_position.xlsx"

# Load Excel files
header_df = pd.read_excel(header_path)
summary_df = pd.read_excel(summary_path)

print("Header file shape:", header_df.shape)
print("Summary file shape:", summary_df.shape)

# Display first few rows
display(header_df.head(3))
display(summary_df.head(3))

Header file shape: (0, 73)
Summary file shape: (7, 2)


,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6)


,Calculated deviation of actual position to nominal position,Unnamed: 1
0,Position,delta Distance (mm)
1,position 1,0.156205
2,position 2,0.163095


In [4]:
# Collect all CSV files
csv_files = glob.glob("../data/raw/sensor_data/*.csv")

print(f"Found {len(csv_files)} CSV files.")

# Load and combine
sensor_dfs = [pd.read_csv(f) for f in csv_files]
sensor_data = pd.concat(sensor_dfs, ignore_index=True)

print("Combined sensor dataset shape:", sensor_data.shape)
sensor_data.head()

Found 18 CSV files.
Combined sensor dataset shape: (153640, 805)


,([13867.472],[-26.880068716264294,-79.91160892471794,57.095391505465344,-157.77176440236227,-105.0096132254245,-44.72477900700451],[-26.881428894723115,-79.91090832767539,57.093360026839825,-157.77315241285316,-105.00685382899981,-44.72477900700451].1,[0.0,0.0,0.0.1,0.0.2,0.0.3,0.0],[0.0.1,0.0.4,-0.0,0.0.5,0.0.6,0.0].1,[2.433830495651467e-17,-2.2138144127730115,-1.5893478394401743,-0.1629906069639269,0.0004506050510721802,0.0].2,[-0.2914353609085083,-2.640852928161621,-2.082642078399658,-0.3690599799156189,-0.1098029613494873,0.0030500823631882668],[0.0.2,0.0.7,0.0.8,0.0.9,0.0.10,0.0].3,[2.850164985011539e-16,-25.67417061620775,-18.441131024188945,-1.3760677172009366,0.0038478213084269736,0.0].4,[-0.2914353609085083.1,-2.64982008934021,-2.080400228500366,-0.3629598021507263,-0.1098029613494873.1,-0.0091502470895648],[-0.6376844833673514,0.27758034088384226,0.7570324633688608,-1.0750504113455182,-1.130220190115434,0.04546741483982896],[1.7623931851118197,-6.732057848264457,-14.279688270286046,0.9177019382926044,0.9781875377348983,0.022847212757561457],[27.37999153137207,28.78999137878418,28.929893493652344,33.39998245239258,33.39385223388672,34.92998123168945]),([12063.671999999999],[-26.93855718662347,-79.65754035991041,57.09543693007859,-157.95641755991915,-105.00957772922095,-44.72474342951372],[-26.94257274817307,-79.65810936611489,57.09472606467383,-157.96049084140827,-105.01376598043987,-44.72614504483851],[-0.0,-0.0.1,[4.5779858540334357e-17,-2.2323120254412623,-1.5923057866060866,-0.16330429136738484,0.00039737376283177444,[-0.5111327767372131,-1.6948087215423584,-1.3540536165237427,0.04727627709507942,-0.10827792435884476,-0.07777710258960724],[5.361102593774372e-16,-25.888692150125458,-18.47545195123562,-1.3787160353404195,0.0033932669604917094,[-0.5178582668304443,-1.7060178518295288,-1.3495699167251587,0.03355090692639351,-0.13877874612808228,-0.09455255419015884],[-0.6397127193662775,0.2794702049695324,0.7547931310799473,-1.0762950380019012,-1.1305776601012318,0.04427250120361925],[3.6642934555815163,-13.701487585498084,-0.855786689260637,0.19293788786696248,1.342352415931503,-1.029739494177423],[27.212703704833984,28.659992218017578,28.59941864013672,33.52998352050781,33.55521011352539,35.32998275756836]),([306.168],57.09539150546534,-157.77176440236232,[-26.88279493255712,-79.91298470518308,-157.76768826151715,-105.01100658401518,-44.72958746018019],[-1.3009621579908202e-17,-2.213814412773011,-1.589347839440174,-0.16299060696392653,0.0004506050510722241,[-0.024659916758537292,-3.371683120727539,-1.546849250793457,-0.38431036472320557,-0.12505337595939636,-0.1662294864654541],[-1.5235065860812838e-16,-25.674170616207746,-18.44113102418894,-1.3760677172009337,0.0038478213084273487,[-0.026901725679636,-3.394101142883301,-1.5401238203048706,-0.388885498046875,-0.12810346484184265,-0.1464039534330368],[-0.6376514909680393,0.2776034348209835,0.7570329763829005,-1.0750878721721475,-1.1302616325684438,0.0453126983717407],[-41.094028857496,11.228906916683105,24.841161461517686,-4.579228925946785,-2.6542899124292636,-3.9239275817378214],[23.18191146850586,24.051803588867188,24.816726684570312,24.939468383789062,24.30146598815918,24.969167709350586]),([12282.599999999999],[-34.19941688221314,-48.11703005422706,57.10107601999953,-180.8795793580347,-105.00517116922576,-44.720326778377384],[-34.20336436422011,-48.113918873164835,57.10087323492684,-180.878425620199,-105.0082471875905,-44.72201961057982],[-5.038883984881787e-17,-4.099610982770684,-1.7217253356556415,-0.2002379817641633,-0.0061885574655097895,[-0.17934484779834747,-3.201305389404297,-1.5087385177612305,0.0366009883582592,-0.050326358526945114,-0.06557676941156387],[-5.900842611227648e-16,-47.544234613547815,-19.97710111945983,-1.6905331392754293,-0.05284553119756084,[-0.17037759721279144,-3.192338228225708,-1.499771237373352,0.030500823631882668,-0.06252668797969818,-0.0762520581483841],[-0.8016982627423562,0.4946531354143632,0.4014577954368928,-1.23041750515295

In [5]:
# Basic info
sensor_data.info()

# Quick statistics
sensor_data.describe().T

# Check for missing values
sensor_data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153640 entries, 0 to 153639
Columns: 805 entries, ([13867.472] to  36.33998107910156])
dtypes: float64(543), object(262)
memory usage: 943.6+ MB


([13867.472]             143455
 [-26.880068716264294     69329
 -79.91160892471794       69329
 57.095391505465344      136015
 -157.77176440236227     133016
                          ...  
 29.80999183654785       146708
 29.569992065429688      146708
 34.59408187866211       146708
 34.67998123168945       146708
 36.33998107910156])     146708
Length: 805, dtype: int64

In [6]:
#Understanding of Degradation Summary
summary_df.describe(include="all").T

,count,unique,top,freq
Calculated deviation of actual position to nominal position,7,7,Position,1
Unnamed: 1,7,7,delta Distance (mm),1


In [8]:
# --- RUN THIS TO INSPECT THE COLUMNS ---

print("--- sensor_data Columns (Case/Spacing is crucial) ---")
print(sensor_data.columns.tolist())
print("\n--- summary_df Columns (Case/Spacing is crucial) ---")
print(summary_df.columns.tolist())

--- sensor_data Columns (Case/Spacing is crucial) ---
['([13867.472]', ' [-26.880068716264294', ' -79.91160892471794', ' 57.095391505465344', ' -157.77176440236227', ' -105.0096132254245', ' -44.72477900700451]', ' [-26.881428894723115', ' -79.91090832767539', ' 57.093360026839825', ' -157.77315241285316', ' -105.00685382899981', ' -44.72477900700451].1', ' [0.0', ' 0.0', ' 0.0.1', ' 0.0.2', ' 0.0.3', ' 0.0]', ' [0.0.1', ' 0.0.4', ' -0.0', ' 0.0.5', ' 0.0.6', ' 0.0].1', ' [2.433830495651467e-17', ' -2.2138144127730115', ' -1.5893478394401743', ' -0.1629906069639269', ' 0.0004506050510721802', ' 0.0].2', ' [-0.2914353609085083', ' -2.640852928161621', ' -2.082642078399658', ' -0.3690599799156189', ' -0.1098029613494873', ' 0.0030500823631882668]', ' [0.0.2', ' 0.0.7', ' 0.0.8', ' 0.0.9', ' 0.0.10', ' 0.0].3', ' [2.850164985011539e-16', ' -25.67417061620775', ' -18.441131024188945', ' -1.3760677172009366', ' 0.0038478213084269736', ' 0.0].4', ' [-0.2914353609085083.1', ' -2.6498200893402

In [7]:
# Example: merging by common columns (adjust names based on actual columns)
common_cols = [col for col in ["TestID", "JointID", "Temperature", "Payload", "Speed"] if col in sensor_data.columns and col in summary_df.columns]

print("Merging on:", common_cols)

merged_df = pd.merge(sensor_data, summary_df, on=common_cols, how="left")
print("Merged dataset shape:", merged_df.shape)

merged_df.head()

Merging on: []


IndexError: list index out of range

In [ ]:
# Example plots — adjust variable names after inspecting your data columns

plt.figure(figsize=(8,5))
sns.histplot(summary_df['DeviationXYZ'], kde=True)
plt.title("Distribution of Positional Deviation (XYZ)")
plt.xlabel("Deviation (mm)")
plt.ylabel("Frequency")
plt.show()

# Example: relationship between temperature and mean deviation
plt.figure(figsize=(8,5))
sns.scatterplot(data=summary_df, x="Temperature", y="DeviationXYZ", hue="Payload", palette="viridis")
plt.title("Temperature vs Positional Deviation")
plt.show()

In [ ]:
#Correlation Heatmap (Numerical Relationships)
plt.figure(figsize=(10,6))
sns.heatmap(sensor_data.corr(numeric_only=True), cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap (Sensor Data)")
plt.show()

# Observation Summary

**Key Insights:**
- Data collected at 125Hz across multiple environmental and operational setups.
- Degradation indicators (position deviation, torque, drift) increase slightly with cycle count.
- Temperature and payload show clear influence on positional accuracy.
- Several missing or noisy samples exist in sensor data, which may require filtering or smoothing.

**Next Steps:**
1. Perform feature engineering (degradation rate, vibration variance, torque fluctuation).
2. Apply hypothesis testing to validate relationships.
3. Prepare the cleaned dataset for predictive modeling in Phase 2.
